# Center-TBI statistics (from XNAT web interface export)
Short description to add here...

Please edit login.cfg with your credentials before executing this script.

Before (re-)running this script, please clear output, shutdown and relaunch kernel, close down and reopen your browser, and then (re-)launch all the cells! Else the memory is not correctly freed (this is a bug in ipywidgets or jupyter notebook).


In [ ]:
%load_ext autoreload
%autoreload 2
# BEWARE: autoreload works on functions and on general code, but NOT on new class methods:
# if you add or change the name of a method, you have to reload the kernel!
# also it will fail if you use super() calls in the classes you change

# Profilers:
# http://pynash.org/2013/03/06/timing-and-profiling/
# http://mortada.net/easily-profile-python-code-in-jupyter.html
# use %lprun -m module func(*args, **kwargs)
%load_ext line_profiler
%load_ext memory_profiler

import pandas as pd

# Setup some display options for pandas
pd.set_option('max_columns', 40)
pd.set_option('expand_frame_repr', False)

In [ ]:
# Edit the filepath to the csv files here
subjects_filepath = 'db/lrq3000_8_6_2017_5_39_9-subjects.csv'
mrisessions_filepath = 'db/lrq3000_8_6_2017_5_40_41-mrsessions.csv'

In [ ]:
# Load the csv into a Pandas DataFrame
sdf = pd.read_csv(subjects_filepath)
mdf = pd.read_csv(mrisessions_filepath)

In [ ]:
# Show structure of sdf
print('Columns for subjects dataframe: ')
print(sdf.columns)
sdf  # to pretty print, just put the dataframe name as the last line, without print or anything

In [ ]:
# Show structure of mdf
print('Columns for mri sessions dataframe: ')
print(mdf.columns)
mdf